In [334]:
import pymysql
import query
import pandas as pd
import seaborn as sns
import sidetable

First things first - read the files


In [335]:
df = pd.read_csv("/home/vishal/ftp_files_csv/bt_attr.csv",usecols=["setting_app_id","blp_score"])

In [336]:
df.head()

,setting_app_id,blp_score
0,10660872622,298
1,10661291196,254
2,10662080444,295
3,10662493120,298
4,10664419139,227


Rename columns for join

In [337]:
scoredf=df.rename(columns={"setting_app_id": "LoanId", "blp_score": "score"})

We need to query the DB in the below date range.

In [338]:
dbquerydf = query.iloans("SELECT * FROM view_FCL_Loan WHERE OriginationDate between '2019-01-01' AND '2020-04-30'")

In [339]:
dbquerydf[["LoanId","IsFirstDefault","OriginationDate"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16507 entries, 0 to 16506
Data columns (total 3 columns):
LoanId             16507 non-null float64
IsFirstDefault     16265 non-null object
OriginationDate    16507 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 387.0+ KB


In [340]:
dbdf=dbquerydf[["LoanId","IsFirstDefault"]]

In [341]:
dbdf.head()

,LoanId,IsFirstDefault
0,2.268087e+10,True
1,2.663282e+10,True
2,1.866559e+10,True
3,1.663031e+10,True
4,1.363352e+10,True


Putting it all together : Left merge dbdf with scoredf on LoanId

In [342]:
view_df = pd.merge(scoredf, dbdf, how ='left', on ='LoanId') 

In [343]:
view_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42484 entries, 0 to 42483
Data columns (total 3 columns):
LoanId            42484 non-null int64
score             42484 non-null int64
IsFirstDefault    4037 non-null object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [344]:
view_df.head()

,LoanId,score,IsFirstDefault
0,10660872622,298,NaN
1,10661291196,254,NaN
2,10662080444,295,NaN
3,10662493120,298,False
4,10664419139,227,NaN


Seeing whether the loan has been lender approved

In [345]:
query_lender_approved='''
select LoanId, 
LoanPrincipal AS ApprovedLoanAmount,
LoanStatus AS LenderApproved 
from view_FCL_Loan_History
where LoanStatus = 'Lender Approved' 
and TimeAdded >= '2019-01-01'
AND TimeAdded <= '2020-04-30'
ORDER BY LoanId , TimeAdded DESC
'''

In [346]:
dblender_approved_df = query.iloans(query_lender_approved)

In [347]:
dblender_approved_df.head()

,LoanId,ApprovedLoanAmount,LenderApproved
0,1.062772e+10,400.0,Lender Approved
1,1.062772e+10,400.0,Lender Approved
2,1.062804e+10,300.0,Lender Approved
3,1.062804e+10,300.0,Lender Approved
4,1.062804e+10,300.0,Lender Approved


In [348]:
view2_df = pd.merge(view_df, dblender_approved_df, how ='left', on ='LoanId') 
view2_df["IsFirstDefault"]*=1
view2_df['Decile_rank'] = pd.qcut(view2_df['score'], 10,labels = False) 
view2_df['ApprovedLoanAmount'] = view2_df['ApprovedLoanAmount'].fillna(0)
view2_df["LenderApproved"] = view2_df["LenderApproved"].replace(to_replace ="Lender Approved", value =1) 
view2_df["LenderApproved"] = view2_df["LenderApproved"].fillna(0)

In [349]:
view2_df.head()

,LoanId,score,IsFirstDefault,ApprovedLoanAmount,LenderApproved,Decile_rank
0,10660872622,298,NaN,0.0,0.0,9
1,10661291196,254,NaN,0.0,0.0,0
2,10662080444,295,NaN,0.0,0.0,7
3,10662493120,298,0,300.0,1.0,9
4,10662493120,298,0,300.0,1.0,9


In [350]:
view2_df.describe()

,LoanId,score,ApprovedLoanAmount,LenderApproved,Decile_rank
count,5.656700e+04,56567.000000,56567.000000,56567.000000,56567.000000
mean,5.289219e+10,280.839288,116.816276,0.340181,4.306504
std,2.579737e+10,30.421763,190.444945,0.473774,2.768189
min,1.066011e+10,108.000000,0.000000,0.000000,0.000000
25%,3.066559e+10,281.000000,0.000000,0.000000,2.000000
50%,5.166608e+10,291.000000,0.000000,0.000000,4.000000
75%,7.467591e+10,295.000000,250.000000,1.000000,7.000000
max,9.968366e+10,299.000000,5000.000000,1.000000,9.000000


In [351]:
view2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56567 entries, 0 to 56566
Data columns (total 6 columns):
LoanId                56567 non-null int64
score                 56567 non-null int64
IsFirstDefault        14801 non-null object
ApprovedLoanAmount    56567 non-null float64
LenderApproved        56567 non-null float64
Decile_rank           56567 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 3.0+ MB


In [352]:
dropnaview2 = view2_df.dropna()
sums1 = []
sums = []
counts = []
for x in range(0,10):
    sums.append(view2_df[view2_df["Decile_rank"]==x].sum()["LenderApproved"])
    counts.append(view2_df[view2_df["Decile_rank"]==x].count()["LenderApproved"])
    sums1.append(dropnaview2[dropnaview2["Decile_rank"]==x].sum()["IsFirstDefault"])

In [353]:
deciledf = pd.DataFrame(list(zip(range(0,10),sums,sums1,counts)), columns =['Decile','LenderApprovedCount','IsFirstDefaultCount','TotalLoansCount']) 

Caveats: I had to consider only the loans for which loans had "IsFirstDefault" Data, around 15k/56k loans.
The data I've pulled seems to heavily skew towards high scores - look at the distribution of score values!

In [354]:
avg = []
for x in range(0,10):
    avg.append((deciledf['IsFirstDefaultCount'][x]/deciledf['TotalLoansCount'][x])*100)

In [355]:
deciledf["PercentDefault"]=avg

In [356]:
templist = []
sum = 0
for x in range(0,10):
    sum += (deciledf["IsFirstDefaultCount"][x]/deciledf["IsFirstDefaultCount"].sum())*100
    templist.append(sum)

In [357]:
deciledf["CumulativePercent"]= templist

In [358]:
deciledf["LenderPercent"]=(deciledf["LenderApprovedCount"]/deciledf["TotalLoansCount"])*100

In [359]:
testdeciledf=sorted(set(pd.qcut(view2_df['score'], 10,retbins= True)[0]))

In [360]:
randomlist = [0,1,2,3,4,5,6,7,8,9]

In [361]:
testdfdecile = pd.DataFrame(data = randomlist, columns = ["Decile"])

In [362]:
testdfdecile["Bins"] = testdeciledf

## Preparing the final table

In [363]:
finaldf = pd.merge(deciledf, testdfdecile, how ='left', on ='Decile') 

In [364]:
cols = list(finaldf.columns.values)

In [365]:
cols = ['Bins','Decile','TotalLoansCount','LenderApprovedCount','LenderPercent','IsFirstDefaultCount','PercentDefault','CumulativePercent']

In [366]:
finaldf=finaldf.reindex(columns=cols)
finaldf.columns = ['Score Range', 'Decile','NumberOfApps',"AppsApproved","FractionAppsApproved","AppsFirstDefault","FractionDefault","CumulativeDefaultFraction"]

In [367]:
finaldf.round(1)

,Score Range,Decile,NumberOfApps,AppsApproved,FractionAppsApproved,AppsFirstDefault,FractionDefault,CumulativeDefaultFraction
0,"(107.999, 260.0]",0,5747,2381.0,41.4,1656.0,28.8,72.2
1,"(260.0, 278.0]",1,6147,983.0,16.0,47.0,0.8,74.2
2,"(278.0, 284.0]",2,5333,1101.0,20.6,43.0,0.8,76.1
3,"(284.0, 288.0]",3,6127,1684.0,27.5,35.0,0.6,77.6
4,"(288.0, 291.0]",4,6744,2167.0,32.1,89.0,1.3,81.5
5,"(291.0, 293.0]",5,6216,2296.0,36.9,112.0,1.8,86.4
6,"(293.0, 294.0]",6,3663,1460.0,39.9,77.0,2.1,89.8
7,"(294.0, 296.0]",7,8404,3533.0,42.0,140.0,1.7,95.9
8,"(296.0, 297.0]",8,4124,1749.0,42.4,69.0,1.7,98.9
9,"(297.0, 299.0]",9,4062,1889.0,46.5,26.0,0.6,100.0
